<a href="https://colab.research.google.com/github/Recommendation-System-Project/Recommendation-system/blob/main/Propensities_Coat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [7]:
item_features = np.genfromtxt("data/item_features.ascii", dtype=None)
user_features = np.genfromtxt("data/user_features.ascii", dtype=None)
train = np.genfromtxt("data/train.ascii", dtype=None)
test = np.genfromtxt("data/test.ascii", dtype=None)

In [9]:
print("item feature shape: ", item_features.shape)
print("user feature shape: ", user_features.shape)
print("train shape: ", train.shape)
print("test shape: ", test.shape)

item feature shape:  (300, 33)
user feature shape:  (290, 14)
train shape:  (290, 300)
test shape:  (290, 300)


## Auto encode item features

In [20]:
item_gender = item_features[:,:2]
item_type = item_features[:,2:18]
item_color = item_features[:,18:31]
item_promot = item_features[:,31:]
print("item_gender shape: ", item_gender.shape)
print("item_type shape: ", item_type.shape)
print("item_color shape: ", item_color.shape)
print("item_promot shape: ", item_promot.shape)

In [33]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape

Auto encode item type

In [ ]:
# Auto encode item type
X_train = item_type
encode_dim = 10
 
## Encoder
encoder = Sequential()
# encoder.add(Flatten(input_shape=[21,]))
encoder.add(Dense(X_train.shape[1]))
encoder.add(Dense(400,activation="relu"))
encoder.add(Dense(200,activation="relu"))
encoder.add(Dense(100,activation="relu"))
encoder.add(Dense(50,activation="relu"))
encoder.add(Dense(encode_dim,activation="relu"))
 
## Decoder
decoder = Sequential()
decoder.add(Dense(50,input_shape=[encode_dim],activation='relu'))
decoder.add(Dense(100,activation='relu'))
decoder.add(Dense(200,activation='relu'))
decoder.add(Dense(400,activation='relu'))
decoder.add(Dense(X_train.shape[1], activation="relu"))
# decoder.add(Reshape([943, 21]))
 
## Autoencoder
autoencoder = Sequential([encoder,decoder])
autoencoder.compile(loss="mse")
autoencoder.fit(X_train,X_train,epochs=50)
 
encoded_item_type = encoder.predict(X_train)

16 -> 10

In [35]:
encoded_item_type.shape

(300, 10)

Auto encode item_color

In [ ]:
# Auto encode item type
X_train = item_color
encode_dim = 7
 
## Encoder
encoder = Sequential()
# encoder.add(Flatten(input_shape=[21,]))
encoder.add(Dense(X_train.shape[1]))
encoder.add(Dense(400,activation="relu"))
encoder.add(Dense(200,activation="relu"))
encoder.add(Dense(100,activation="relu"))
encoder.add(Dense(50,activation="relu"))
encoder.add(Dense(encode_dim,activation="relu"))
 
## Decoder
decoder = Sequential()
decoder.add(Dense(50,input_shape=[encode_dim],activation='relu'))
decoder.add(Dense(100,activation='relu'))
decoder.add(Dense(200,activation='relu'))
decoder.add(Dense(400,activation='relu'))
decoder.add(Dense(X_train.shape[1], activation="relu"))
# decoder.add(Reshape([943, 21]))
 
## Autoencoder
autoencoder = Sequential([encoder,decoder])
autoencoder.compile(loss="mse")
autoencoder.fit(X_train,X_train,epochs=50)
 
encoded_item_color = encoder.predict(X_train)

14 -> 7

In [40]:
encoded_item_color.shape

(300, 7)

## Join item features

In [41]:
item_features_encoded = np.concatenate((item_gender, encoded_item_type),axis = 1)
item_features_encoded = np.concatenate((item_features_encoded, encoded_item_color),axis = 1)
item_features_encoded = np.concatenate((item_features_encoded, item_promot),axis = 1)
item_features_encoded

array([[1.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [1.        , 0.        , 0.        , ..., 0.24560076, 0.        ,
        1.        ],
       [1.        , 0.        , 0.52863282, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 1.        , 1.06302667, ..., 0.22257644, 0.        ,
        1.        ],
       [0.        , 1.        , 0.        , ..., 0.22257644, 0.        ,
        1.        ],
       [0.        , 1.        , 1.06302667, ..., 0.        , 0.        ,
        1.        ]])

33 -> 21

In [42]:
item_features_encoded.shape

(300, 21)

## Joint user-item pair features

Concatenate user and item features into a matrix for all possible user item pairs

In [46]:
O_mask = np.int64(train > 0)
O_mask

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

creat array that contains features of all possible user-item pairs

47 -> 35

In [49]:
features_matrix = []
for u in range(O_mask.shape[0]):
  for i in range(O_mask.shape[1]):
    user_f = user_features[u]
    item_f = item_features_encoded[i]
    observed = np.concatenate((user_f, item_f))
    features_matrix.append(observed)
features_matrix = np.array(features_matrix)
features_matrix.shape

(87000, 35)

## Log Reg to predict propensity based on features

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
x = features_matrix
y = np.ndarray.flatten(O_mask)

In [61]:
x.shape

(87000, 35)

In [62]:
model = LogisticRegression(max_iter=1000).fit(x,y)
model.classes_, model.intercept_, model.coef_

(array([0, 1]),
 array([-1.90280115]),
 array([[-0.01239614, -0.01234366, -0.00414676, -0.00415704, -0.00413256,
         -0.00418339, -0.00435492, -0.00376514, -0.00823546, -0.00826195,
         -0.00824239, -0.00825385, -0.00825889, -0.00822707,  0.14296884,
         -0.16770865, -0.42456468, -0.12126716, -0.60191854, -1.21092114,
         -0.00730095,  0.39214212,  0.52094083,  0.28665924, -0.26085132,
         -0.55749375,  0.34669588,  0.04873553,  1.20028726,  0.13905638,
          0.31480502,  1.2298091 , -0.12594555,  0.61283535, -0.63757516]]))

In [63]:
predicted_propensity = model.predict_proba(x)
predicted_propensity

array([[0.72827884, 0.27172116],
       [0.92598156, 0.07401844],
       [0.92374828, 0.07625172],
       ...,
       [0.92752951, 0.07247049],
       [0.90603049, 0.09396951],
       [0.93905633, 0.06094367]])

In [64]:
predicted_propensity = predicted_propensity[:,1].reshape(290, 300)
predicted_propensity

array([[0.27172116, 0.07401844, 0.07625172, ..., 0.07247618, 0.09397671,
        0.06094851],
       [0.27170996, 0.07401456, 0.07624773, ..., 0.07247237, 0.09397189,
        0.06094527],
       [0.27171199, 0.07401526, 0.07624845, ..., 0.07247306, 0.09397277,
        0.06094586],
       ...,
       [0.27170548, 0.07401301, 0.07624613, ..., 0.07247085, 0.09396997,
        0.06094398],
       [0.27170442, 0.07401264, 0.07624576, ..., 0.07247049, 0.09396951,
        0.06094367],
       [0.27170442, 0.07401264, 0.07624576, ..., 0.07247049, 0.09396951,
        0.06094367]])

In [65]:
model.score(x, y)

0.92

In [66]:
np.savetxt("our_propensities.ascii", predicted_propensity, delimiter=" ")

In [70]:
np.genfromtxt("our_propensities.ascii", dtype=None).shape

(290, 300)